# Import Libraries, Functions, and Data

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import zipfile
from scipy.stats import percentileofscore

## Define Functions

In [2]:
def tackles_dataset(tackle_filepath, players_filepath):
  """
  This function creates an enriched and detailed dataframe making use of
  the relevant fields in the players and tackles datasets
  """
  # Define our tackles dimension table
  tackles = pd.read_csv(tackle_filepath)
  # Create gamePlayId
  tackles['gamePlayId'] = tackles.gameId.astype(str) + tackles.playId.astype(str)
  # Enrich by adding a tackler name and position to nflId
  def add_prefix_to_columns(df, prefix):
      d = df.copy()
      d.columns = [f"{prefix}{col}" for col in d.columns]
      return d
  p = pd.read_csv(players_filepath)
  p = add_prefix_to_columns(p, 'tackle_')
  tackles = tackles.merge(p, how='left', left_on='nflId', right_on='tackle_nflId')
  return tackles

In [3]:
def enrich_tracking_week(filepath, nrows=None):
  """
  This function takes one tracking_week file and adds some relevant context to
  the tracking_week files.
  """
  # Load in additional datasets
  plays = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2024/plays.csv")
  games = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2024/games.csv")
  players = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2024/players.csv")
  # Load in our tracking_week file "t"
  t = pd.read_csv(filepath, nrows=nrows)
  # Merge with relevant columns
  t = t.merge(plays[['gameId', 'playId', 'ballCarrierId', 'ballCarrierDisplayName', 'playDescription',
                    'playResult', 'yardlineNumber', 'yardsToGo', 'yardlineSide',
                    'possessionTeam']],
              how = 'left', on=['gameId', 'playId'])
  t = t.merge(games[['gameId', 'homeTeamAbbr', 'visitorTeamAbbr']],
              how='left', on='gameId')
  t = t.merge(players[['nflId', 'position']], how='left', on='nflId')
  # Add a new column 'rank' based on 'gameId', 'playId', and 'timestamp'
  t['time'] = pd.to_datetime(t['time'])
  t['dense_rank'] = t.groupby(['gameId', 'playId'])['time'].rank(method='dense').astype(int)
  return t

In [4]:
def pbp_df(gameId, playId, week_num=1):
  """
  This function filters a large tracking_week dataframe down to just one
  specific play that will be used in the play_by_play function.
  """
  p = enrich_tracking_week(f"/kaggle/input/nfl-big-data-bowl-2024/tracking_week_{week_num}.csv")
  p = p[(p.gameId == gameId) & (p.playId == playId)]
  p = p.sort_values('time')
  return p

In [5]:
def play_by_play(p, save=False):
  """
  This function creates an animated gif of a given gameId and playId (pass in
  the pbp_df() function to make this work simply).
  """
  import pandas as pd
  import matplotlib.pyplot as plt
  import numpy as np
  from matplotlib.animation import FuncAnimation
  import textwrap

  plt.close("all")
  # Number of frames (dependent on the amount of time stamps during each play)
  N_times = p.dense_rank.max()
  # Number of groups (11 for each team plus the football will always be 23)
  N_tracks = 23

  # Create the data to be plotted
  df = p.copy()
  df = df.sort_values('time')
  df = df[['frameId', 'time', 'nflId', 'x', 'y', 'o', 'club', 'jerseyNumber', 'homeTeamAbbr',
          'visitorTeamAbbr', 'ballCarrierDisplayName', 'playDescription', 'position',
          'playResult', 'yardlineNumber', 'yardsToGo', 'yardlineSide', 'possessionTeam']]
  # Define conditions and corresponding values
  condition1 = (df['club'] == df['visitorTeamAbbr'])
  condition2 = (df['club'] == df['homeTeamAbbr'])
  condition3 = (df['club'] == 'football')
  # Define corresponding values for each condition
  values = ['#BBBBFF', '#FFBBBB', '#FFFF00']
  # Use numpy's select function to apply conditions
  df['Color'] = np.select([condition1, condition2, condition3], values, default='Other')
  # Add avg_o_pos and avg_d_position
  df['avgDPos'] = df[(df.frameId == 1) & (df.position.isin(['SS', 'FS', 'CB']))]['x'].mean()
  df['avgOPos'] = df[(df.frameId == 1) & (df.position.isin(['T', 'G', 'C', 'QB']))]['x'].mean()
  # Identify the line of scrimmage based on the 0 to 120 axis
  def calculate_graph_lineofscrimmage(row):
      if (row['avgOPos'] < row['avgDPos']) and (row['possessionTeam'] == row['yardlineSide']):
        return 10 + row['yardlineNumber']
      elif (row['avgOPos'] < row['avgDPos']) and (row['possessionTeam'] != row['yardlineSide']):
        return 60 + (50 - row['yardlineNumber'])
      elif (row['avgOPos'] > row['avgDPos']) and (row['possessionTeam'] == row['yardlineSide']):
        return 110 - row['yardlineNumber']
      elif (row['avgOPos'] > row['avgDPos']) and (row['possessionTeam'] != row['yardlineSide']):
        return 10 + row['yardlineNumber']
      else:
        return None
  df['GraphLoS'] = df.apply(calculate_graph_lineofscrimmage, axis=1)
  # Create a first down line based on a 0 to 120 axis
  def calculate_firstdown_marker(row):
    if (row['avgOPos'] < row['avgDPos']) and (row['possessionTeam'] == row['yardlineSide']):
      return row['GraphLoS'] + row['yardsToGo']
    elif (row['avgOPos'] < row['avgDPos']) and (row['possessionTeam'] != row['yardlineSide']) and (row['GraphLoS'] >= 10):
      return row['GraphLoS'] + row['yardsToGo']
    elif (row['avgOPos'] > row['avgDPos']) and (row['possessionTeam'] == row['yardlineSide']):
      return row['GraphLoS'] - row['yardsToGo']
    elif (row['avgOPos'] > row['avgDPos']) and (row['possessionTeam'] == row['yardlineSide']) and (row['GraphLoS'] >= 10):
      return row['GraphLoS'] - row['yardsToGo']
    else:
      return None
  df['GraphFirstDown'] = df.apply(calculate_firstdown_marker, axis=1)
  # Replace null nflIds with 'ball' to represent the football
  df['nflId'] = df['nflId'].replace(np.nan, 'ball')
  # Replace the nans with blanks so nothing will show up on the animation
  df['jerseyNumber'] = df['jerseyNumber'].replace(np.nan, '')
  df['jerseyNumber'] = df['jerseyNumber'].astype(str).str.split('.').str[0]
  df = df.drop(['time', 'yardlineNumber', 'yardsToGo'], axis=1)
  df = df.rename(columns={'nflId':'Track', 'x':'X-Position', 'y':'Y-Position', 'o':"Orientation",
                          'club':'Club', 'jerseyNumber':'JerseyNumber'})

  # Create the plotting area
  fig, ax = plt.subplots()
  ax.set_xlim(0, 120)
  ax.set_ylim(0, 50)
  ax.axis(False)
  # Add background fill color
  ax.axvspan(0, 10, facecolor='navy', alpha=0.5, zorder=1)
  ax.axvspan(10, 110, facecolor='lightgreen', alpha=0.5, zorder=1)
  ax.axvspan(110, 120, facecolor='navy', alpha=0.5, zorder=1)
  # Add vertical lines for end zones
  ax.axvline(10, linewidth=2, color='white', zorder=2)
  ax.axvline(110, linewidth=2, color='white', zorder=2)
  # Add text in the end zone
  ax.text(5, 25, f'Home Team ({df["homeTeamAbbr"].iloc[0]})',
          fontsize=14, ha='center', va='center', rotation=90,
          fontweight='bold', color='yellow', zorder=2)
  ax.text(115, 25, f'Visiting Team ({df["visitorTeamAbbr"].iloc[0]})',
          fontweight='bold', fontsize=14, ha='center', va='center',
          rotation=270, color='yellow', zorder=2)
  # Add vertical lines for every 10 yards on the field
  for i in range(20, 110, 10):
    zorder = 2
    # Calculate the yardage to show on the field
    if i <= 60:
      field_number = (i - 10)
    else:
      field_number = 110 - i
    # Add yard marker every ten yards
    ax.axvline(i, linewidth=1, color='white', zorder=zorder)
    # Add field numbers are the bottom
    ax.text(i - 0.8, 5, int(field_number/10), fontsize=8, fontweight='bold',
            color='white', ha='right', va='bottom', zorder=zorder)
    ax.text(i + 0.8, 5, "0", fontsize=8, fontweight='bold',
            color='white', ha='left', va='bottom', zorder=zorder)
    # Add field numbers at the top
    ax.text(i + 2.6, 45, int(field_number/10), fontsize=8, fontweight='bold', ha='right',
            color='white', va='top', zorder=zorder, rotation=180)
    ax.text(i - 2.6, 45, "0", fontsize=8, fontweight='bold', ha='left',
            color='white', va='top', zorder=zorder, rotation=180)
  # Add vertical lines at the positions where 'GraphLoS' and 'GraphFirstDown' are maximized
  ax.axvline(df['GraphLoS'].max(), linewidth=1.5, color='navy', zorder=3)
  ax.axvline(df['GraphFirstDown'].max(), linewidth=1.5, color='yellow', zorder=3)
  # Set title
  title = f"{df['playDescription'].iloc[0]} Play resulted in a {df['playResult'].iloc[0]} yard gain."
  wrapped_title = textwrap.fill(title, width=60)
  ax.set_title(wrapped_title, fontsize=10)

  # Initialize lists for scatters and text annotations
  scatters = []
  annotations = []

  for track, group in df.groupby("Track"):
      scatter = ax.scatter(group["X-Position"].iloc[0],
                          group["Y-Position"].iloc[0],
                          s=45, c=group['Color'].iloc[0], zorder=4)
      scatters.append(scatter)

      text_annotation = ax.text(group["X-Position"].iloc[0],
                                group["Y-Position"].iloc[0],
                                str(group["JerseyNumber"].iloc[0]), ha='center',
                                va='center', fontsize=6, zorder=5)
      annotations.append(text_annotation)

  # Function to update the plot for each frame
  def animate(i):
      for scatter, text_annotation, (_, group) in zip(scatters, annotations, df.groupby("Track")):
          scatter.set_offsets((group["X-Position"].iloc[i],
                              group["Y-Position"].iloc[i]))
          text_annotation.set_position((group["X-Position"].iloc[i],
                                        group["Y-Position"].iloc[i]))
          text_annotation.set_text(str(group["JerseyNumber"].iloc[i]))

  # Render and return the video
  from matplotlib import rc

  anim = FuncAnimation(fig, animate, frames=N_times, interval=250)
  plt.close(fig)
  rc('animation', html='html5')
  if save:
    anim.save("play_by_play.gif", writer="pillow")
  return anim

# About

&emsp;A defender involved in the play either made the tackle, assisted in the tackle, or missed the tackle. Looking at these statistics alone doesn't paint the whole picture. A missed tackle in this context will always seem like a bad thing, when in reality, the quality of a missed tackle lies on a spectrum.

&emsp;In this study, we will look through the foundational building blocks of what makes a missed tackle "good" or "bad", and then how we can measure the quality of a missed tackle as a rating.

**A "Good" Missed Tackle**

Below, we see a good missed tackle from red circle 52 as he was able to turn the ball carrier back inside to his defense in the open field that resulted in minimal to no gain.

In [6]:
# Home team is RED, visiting team is BLUE
play_by_play(pbp_df(2022090800, 1102, 1))

**A "Poor" Missed Tackle**

Below you see red number 9 take a bad angle to the ball carrier, did close to nothing in terms of having a positive effect on the play, and missed a very costly tackle that led to a 74 yard touchdown. This missed tackle is clearly worse than the one above.

In [7]:
# Home team is RED, visiting team is BLUE
play_by_play(pbp_df(2022102309, 3809, week_num=7))

# Breaking Down the Rating System

**Missed Tackle Rating**

Now that we've qualitatively analyzed a couple examples, let's add a quantitative rating to each missed tackle in the dataset provided. We will look at three key factors when assessing the quality of a missed tackle:
1. <u>In Frame Yards Gained Per Second</u> - measures the yards the ball carrier gained from the time the missed tackler gets within two yards of the ball carrier, to the time the ball carrier gets passed the missed tackler at one plus the missed tacklers closest distance to the ball carrier during the play divided by the time it took for the ball carrier to shake the defender.
  * In other words, how many yards the ball carrier gained in terms of the total time it took for the ball carrier to shake the defender.
2. <u>Total Yards Gained After the Missed Tackle</u> - measures the yards gained after the ball carrier shook the missed tackle.
3. <u>Defender Closing Distance</u> - measures how much distance the next closest defender gained to the ball carrier from the time the missed tackler entered and exited the frame of the play.

The rating is calculated as follows:<br>
rating = (-0.5)((x - ux)/sx) + (-0.25)((y - uy)/sy) + (0.25)((z - uz)/sz)

Where...
* <u>x</u> = value for "In Frame Yards Gained Per Second"
* <u>ux</u> = dataset mean for "In Frame Yards Gained Per Second"
* <u>sx</u> = dataset standard deviation for "In Frame Yards Gained Per Second"
* <u>y</u> = value for "Total Yards Gained After the Missed Tackle"
* <u>uy</u> = dataset mean for "Total Yards Gained After the Missed Tackle"
* <u>sy</u> = dataset standard deviation for "Total Yards Gained After the Missed Tackle"
* <u>z</u> = value for "Defender Closing Distance"
* <u>uz</u> = dataset mean for "Defender Closing Distance"
* <u>us</u> = dataset standard deviation for "Defender Closing Distance"

# Scoring Our First Two Examples

In [8]:
# The code that assembled this dataset will be in the appendix
ratings = pd.read_excel('/kaggle/input/2024-nfl-big-data-bowl-mtdf-all-weeks-w-rating/mtdf_all_weeks_w_rating.xlsx')
ratings.head()

,missedTackler,missedTacklerJ,enteredFrame,enteredTime,firstContactFrame,firstContactTime,nextDefender,nextDefenderJ,nextDefenderDistance,exitFrame,...,exitDefenderDistance,totalTime,ydsGainedDuringMissedTackle,totalYdsGainedAfterMissedTackle,gamePlayId,sourceFile,defenderClosingDistance,defenderClosingSpeed,inFrameYdsGainedPerSecond,Rating
0,52492,52,10,2022-09-08 21:03:52.200,14.0,2022-09-08 21:03:52.600,42816,2,5.560872,18,...,0.925419,0.800000,2.22,3.00,20220908001102,tracking_week_1.csv,4.635453,5.794317,2.775000,4.272169
1,40107,23,54,2022-09-08 21:17:52.200,55.0,2022-09-08 21:17:52.300,40166,21,3.281737,74,...,0.911811,2.000000,6.68,6.36,20220908001385,tracking_week_1.csv,2.369926,1.184963,3.340000,3.764190
2,46232,99,37,2022-09-08 21:17:50.500,55.0,2022-09-08 21:17:52.300,44976,58,2.514478,43,...,2.934365,0.599999,4.51,18.78,20220908001385,tracking_week_1.csv,-0.419887,-0.699813,7.516679,2.085775
3,43335,94,38,2022-09-08 22:00:46.000,40.0,2022-09-08 22:00:46.200,47917,91,3.049475,49,...,0.796116,1.099999,5.35,10.11,20220908002163,tracking_week_1.csv,2.253360,2.048511,4.863641,3.285402
4,42816,2,11,2022-09-08 22:02:07.900,16.0,2022-09-08 22:02:08.400,47862,22,9.310258,19,...,4.521172,0.800000,1.73,9.61,20220908002208,tracking_week_1.csv,4.789085,5.986357,2.162500,4.250782


In [9]:
# How did our first two examples stack up against all other missed tackles in
# the first 9 weeks of the 2022 NFL season?
good_play_percentile = percentileofscore(ratings.Rating,
                                         ratings[ratings.gamePlayId == 20220908001102]['Rating'].iloc[0])
good_play_percentile = "{:.1f}".format(good_play_percentile)
bad_play_percentile = percentileofscore(ratings.Rating,
                                        ratings[ratings.gamePlayId == 20221023093809]['Rating'].iloc[0])
bad_play_percentile = "{:.1f}".format(bad_play_percentile)
print(f"""In our 'good' missed tackle example, the rating scored in the {good_play_percentile} percentile,
indicating it was a good play! The rating was {round(ratings[ratings.gamePlayId == 20220908001102]['Rating'].iloc[0], 2)}\n""")
print(f"""In our 'bad' missed tackle example, the rating scored in the {bad_play_percentile} percentile,
indicated it was one of the worst missed tackles, if not the worst. The rating was {round(ratings[ratings.gamePlayId == 20221023093809]['Rating'].iloc[0], 2)}""")

In our 'good' missed tackle example, the rating scored in the 84.3 percentile,
indicating it was a good play! The rating was 4.27

In our 'bad' missed tackle example, the rating scored in the 0.1 percentile,
indicated it was one of the worst missed tackles, if not the worst. The rating was 0.0


# Top Two Highest Quality Missed Tackles

In [10]:
# What were the ratings and statistics for the top 2 best missed tackles?
ratings.sort_values('Rating', ascending=False).head(2)

,missedTackler,missedTacklerJ,enteredFrame,enteredTime,firstContactFrame,firstContactTime,nextDefender,nextDefenderJ,nextDefenderDistance,exitFrame,...,exitDefenderDistance,totalTime,ydsGainedDuringMissedTackle,totalYdsGainedAfterMissedTackle,gamePlayId,sourceFile,defenderClosingDistance,defenderClosingSpeed,inFrameYdsGainedPerSecond,Rating
1699,42116,21,6,2022-10-30 13:12:27.100,9.0,2022-10-30 13:12:27.400,44837,3,17.999003,32,...,1.072007,2.600001,0.21,6.51,2022103007309,tracking_week_8.csv,16.926995,6.510380,0.080769,6.332471
1809,47913,49,15,2022-11-06 13:25:29.900,18.0,2022-11-06 13:25:30.200,41269,8,10.462586,40,...,0.362353,2.500000,-0.25,0.00,2022110600633,tracking_week_9.csv,10.100232,4.040093,-0.100000,5.711407


In [11]:
# Highest quality missed tackle
gamePlayId = 2022103007309
rating = round(ratings[ratings.gamePlayId == gamePlayId]['Rating'].iloc[0] ,2)
totalTime = round(ratings[ratings.gamePlayId == gamePlayId]['totalTime'].iloc[0], 1)
yrdsGainedDuring = round(ratings[ratings.gamePlayId == gamePlayId]['ydsGainedDuringMissedTackle'].iloc[0], 2)
totalYrdsGained = round(ratings[ratings.gamePlayId == gamePlayId]['totalYdsGainedAfterMissedTackle'].iloc[0], 2)
missedTacklerJ = ratings[ratings.gamePlayId == gamePlayId]['missedTacklerJ'].iloc[0]
closing = round(ratings[ratings.gamePlayId == gamePlayId]['defenderClosingDistance'].iloc[0], 2)
print(f"""Number {missedTacklerJ} missed the tackle on the play holding the ball carrier to {yrdsGainedDuring} yards
thoughout the duration of the missed tackle taking {totalTime} seconds, where the next closest defender was {closing}
yards closer after the tackle attempt. The ball carrier gained an additional {totalYrdsGained} yards afterwards.
This missed tackle recieved a quality score of {rating} out of {round(ratings.Rating.max(), 2)}.\n\n""")

# Print out the play by play video
play_by_play(pbp_df(2022103007, 309, week_num=8))

Number 21 missed the tackle on the play holding the ball carrier to 0.21 yards
thoughout the duration of the missed tackle taking 2.6 seconds, where the next closest defender was 16.93
yards closer after the tackle attempt. The ball carrier gained an additional 6.51 yards afterwards.
This missed tackle recieved a quality score of 6.33 out of 6.33.




In [12]:
# Second highest quality missed tackle
gamePlayId = 2022110600633
rating = round(ratings[ratings.gamePlayId == gamePlayId]['Rating'].iloc[0] ,2)
totalTime = round(ratings[ratings.gamePlayId == gamePlayId]['totalTime'].iloc[0], 1)
yrdsGainedDuring = round(ratings[ratings.gamePlayId == gamePlayId]['ydsGainedDuringMissedTackle'].iloc[0], 2)
totalYrdsGained = round(ratings[ratings.gamePlayId == gamePlayId]['totalYdsGainedAfterMissedTackle'].iloc[0], 2)
missedTacklerJ = ratings[ratings.gamePlayId == gamePlayId]['missedTacklerJ'].iloc[0]
closing = round(ratings[ratings.gamePlayId == gamePlayId]['defenderClosingDistance'].iloc[0], 2)
print(f"""Number {missedTacklerJ} missed the tackle on the play holding the ball carrier to {yrdsGainedDuring} yards
thoughout the duration of the missed tackle taking {totalTime} seconds, where the next closest defender was {closing}
yards closer after the tackle attempt. The ball carrier gained an additional {totalYrdsGained} yards afterwards.
This missed tackle recieved a quality score of {rating} out of {round(ratings.Rating.max(), 2)}.\n\n""")

# Print out the play by play video
play_by_play(pbp_df(2022110600, 633, week_num=9))

Number 49 missed the tackle on the play holding the ball carrier to -0.25 yards
thoughout the duration of the missed tackle taking 2.5 seconds, where the next closest defender was 10.1
yards closer after the tackle attempt. The ball carrier gained an additional 0.0 yards afterwards.
This missed tackle recieved a quality score of 5.71 out of 6.33.




# Current vs. Next Gen Missed Tackle Statistics

In [13]:
# Under the current system, we look at missed tackles this way
tackles = tackles_dataset('/kaggle/input/nfl-big-data-bowl-2024/tackles.csv', 
                          '/kaggle/input/nfl-big-data-bowl-2024/players.csv')
mt_by_player = tackles.groupby(['tackle_displayName', 'tackle_position'], as_index=False).agg({'pff_missedTackle':'sum'})
mt_by_player = mt_by_player.rename(columns={'tackle_displayName':'playerName', 'tackle_position':'position',
                                            'pff_missedTackle':'sumMissedTackles'})
mt_by_player.sort_values('sumMissedTackles', ascending=False)

,playerName,position,sumMissedTackles
346,Jalen Pitre,FS,19
663,Rayshawn Jenkins,SS,16
797,Zaven Collins,OLB,13
255,Donovan Wilson,SS,13
262,Drue Tranquill,ILB,13
...,...,...,...
184,Daniel Wise,DE,0
635,Payton Turner,DE,0
636,Percy Butler,FS,0
424,Joseph Ossai,DE,0


In [14]:
# Calculate the sum of missed tackles by position
mt_by_player.groupby('position', as_index=False).agg({'sumMissedTackles':'sum'}).sort_values('sumMissedTackles', ascending=False)

,position,sumMissedTackles
0,CB,518
8,OLB,330
5,ILB,329
9,SS,257
4,FS,213
2,DE,191
3,DT,182
7,NT,41
6,MLB,23
1,DB,6


<u>**NOTE:**</u> under the current system, we would simply conclude Jalen Pitre is the worst tackler in the NFL, and cornerbacks are the worst tackling position. Of course, we could look at percentage of missed tackles per game, per play on the field, etc. since all that data is made available before AWS' involvment with the NFL. However, let's keep this example simple and look only at total number of missed tackles.

In [15]:
# Create a dataframe of missed tackles by player using next gen missed tackle stats
mt_stats = ratings.groupby('missedTackler', as_index=False).agg({'ydsGainedDuringMissedTackle':'mean',
                                                      'totalTime':'mean', 'defenderClosingDistance':'mean',
                                                      'totalYdsGainedAfterMissedTackle':'mean',
                                                      'inFrameYdsGainedPerSecond':'mean',
                                                      'Rating':'mean', 'gamePlayId':'count'})
players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/players.csv')
mt_stats = mt_stats.merge(players[['displayName', 'position', 'nflId']],
                          how='left', left_on='missedTackler', right_on='nflId')
mt_stats = mt_stats.drop(['nflId', 'missedTackler'], axis=1)
mt_stats = mt_stats.iloc[:,[7,8,6,0,1,4,2,3,5]]
mt_stats.columns = mt_stats.columns[:2].tolist() + ['avg_' + col for col in mt_stats.columns[2:]]
mt_stats = mt_stats.rename(columns={'avg_gamePlayId':'countMissedTackles'})
print(f"Highest Rating by Player: {mt_stats.avg_Rating.max()}")
print(f"Average Rating by Player: {mt_stats.avg_Rating.mean()}")
print(f"Lowest Rating by Player: {mt_stats.avg_Rating.min()}\n")
mt_stats[mt_stats.countMissedTackles >= 5].sort_values('avg_Rating', ascending=False).head(10)

Highest Rating by Player: 4.972203055463455
Average Rating by Player: 3.579798188530732
Lowest Rating by Player: 1.2396781104646122



,displayName,position,countMissedTackles,avg_ydsGainedDuringMissedTackle,avg_totalTime,avg_inFrameYdsGainedPerSecond,avg_defenderClosingDistance,avg_totalYdsGainedAfterMissedTackle,avg_Rating
536,Martin Emerson,CB,7,0.392857,0.914286,1.401562,2.758844,2.311429,4.382927
524,Roger McCreary,CB,6,0.346667,0.800000,0.969580,2.104396,4.326667,4.347374
25,Casey Hayward,CB,5,1.100000,1.040000,1.241587,2.160493,4.552000,4.283852
530,Alontae Taylor,CB,6,0.491667,0.766667,1.260017,1.328114,3.050000,4.218392
531,Troy Andersen,ILB,5,1.254000,1.100001,1.821363,2.790241,5.572000,4.196569
343,Michael Jackson,CB,5,1.476000,1.040000,1.726860,2.262984,4.710000,4.177651
20,Chandler Jones,DE,5,1.296000,0.940000,1.127880,0.724325,3.158000,4.171734
374,A.J. Terrell,CB,8,0.670000,0.887500,1.244965,0.789625,5.150000,4.096477
327,Quincy Williams,OLB,9,1.091111,0.700000,1.779033,2.104612,7.292222,4.073483
329,Chauncey Gardner-Johnson,SS,10,1.491000,0.820000,1.991044,2.440050,8.472000,4.032024


In [16]:
# Next gen missed tackle stats by position
stats_by_pos = mt_stats.groupby('position', as_index=False).agg({'countMissedTackles':'sum',
                                                  'avg_ydsGainedDuringMissedTackle':'mean',
                                                  'avg_totalTime':'mean',
                                                  'avg_inFrameYdsGainedPerSecond':'mean',
                                                  'avg_defenderClosingDistance':'mean',
                                                  'avg_totalYdsGainedAfterMissedTackle':'mean',
                                                  'avg_Rating':'mean'}).rename(columns={'avg_Rating':'avg_PosRating'})
stats_by_pos.sort_values('avg_PosRating', ascending=False)

,position,countMissedTackles,avg_ydsGainedDuringMissedTackle,avg_totalTime,avg_inFrameYdsGainedPerSecond,avg_defenderClosingDistance,avg_totalYdsGainedAfterMissedTackle,avg_PosRating
1,DB,5,1.432000,0.719999,2.333249,2.398783,6.214000,4.009350
0,CB,511,2.522871,0.873303,2.991801,1.652697,8.955386,3.684863
2,DE,166,2.784793,0.989265,2.818598,0.880394,8.342389,3.647184
3,DT,160,2.961571,0.982586,3.049927,0.892373,8.671143,3.584899
8,OLB,304,2.985476,1.000458,3.157021,1.270587,9.492893,3.583506
5,ILB,314,2.906174,0.944219,3.364965,1.510128,9.700938,3.558309
9,SS,247,2.756097,0.864171,3.358688,1.378398,10.091282,3.532549
7,NT,40,3.117841,1.018182,3.449631,1.033745,9.136818,3.495094
4,FS,209,3.123625,0.855633,3.731903,0.912626,12.011991,3.333065
6,MLB,22,4.003976,0.977857,3.908128,0.829721,11.270619,3.301960


<u>**NOTE:**</u> with this context in mind we see that with a minimum of five missed tackles, Martin Emerson produces some of the highest quality missed tackles. Even if he isn't making the play, he's still contributing at a high level.

# Summary

&emsp;In summary, there's more to missed tackling than simply counting how many a player commits. In fact, there's more to tackling than summing total tackles, assists, tackles for loss, missed tackles, etc. Just as quarterbacks have a QBR system, defenses can have a TR (Tackle Rating) that goes beyond just scoring the quality of missed tackles but also the quality of tackles made. Let this concept serve as the foundation of moving towards a Tackle Rating system where the next generation of tackling statistics will prove to be more accurate and comprehensive.

# Appendix

## Extracting In-Depth Missed Tackle Data



In [17]:
# Define input dataframe (could be from any week)
t = enrich_tracking_week('/kaggle/input/nfl-big-data-bowl-2024/tracking_week_1.csv')

def tackle_tracking(t):
  dis = t.copy()
  ball_carrier_xy = dis[dis.nflId == dis.ballCarrierId].groupby(['gameId', 'playId', 'time'],as_index=False)[['gameId', 'playId', 'time', 'x', 'y']].transform('max')
  dis = dis.merge(ball_carrier_xy, how='left', on=['gameId', 'playId', 'time'])
  dis = dis.rename(columns={'x_y':'ballCarrierX', 'y_y':'ballCarrierY',
                            'x_x':'x', 'y_x':'y'})
  # Function to calculate Euclidean distance
  def calculate_distance(row):
      x_diff = row['x'] - row['ballCarrierX']
      y_diff = row['y'] - row['ballCarrierY']
      return np.sqrt(x_diff**2 + y_diff**2)
  dis['distance_to_ballcarrier'] = dis.apply(calculate_distance, axis=1)
  dis['gamePlayId'] = dis.gameId.astype(str) + dis.playId.astype(str)
  return dis

In [18]:
# Define input dataframes
tracking = tackle_tracking(enrich_tracking_week('/kaggle/input/nfl-big-data-bowl-2024/tracking_week_1.csv'))
tt = tackles_dataset('/kaggle/input/nfl-big-data-bowl-2024/tackles.csv', 
                     '/kaggle/input/nfl-big-data-bowl-2024/players.csv')[['gamePlayId', 'tackle_displayName',
                                                    'nflId', 'tackle', 'assist', 'pff_missedTackle']]

# Create function to extract additional information that be used in the rating score
def process_missed_tackles(tracking, tt, gamePlayId):
  # Define our dataframe f
  f = tracking.merge(tt, how='left', on=['gamePlayId', 'nflId'])
  temp = f[f.gamePlayId == gamePlayId]

  # Find the average offense and defense positions that will be used later in the script
  offense_pos = temp[(temp.position.isin(['T', 'G', 'C', 'QB'])) & (temp.frameId == 1)]['x'].mean()
  defense_pos = temp[(temp.position.isin(['SS', 'FS', 'CB'])) & (temp.frameId == 1)]['x'].mean()

  # Find the minimum index where the ball is less than 2 yards to the ball carrier
  ball = temp[temp.club == 'football']
  min_frame_football = ball[ball['distance_to_ballcarrier'] <= 2.0]['frameId'].min()

  # Overwrite temp to filter out irrelevant indices
  temp = temp[temp.index >= min_frame_football]
  # Filter so we are only looking at defensive players
  temp = temp[(temp.club != temp.possessionTeam) & (temp.club != 'football')]

  # Identify our player who missed the tackle
  missed_tacklers = temp[temp.pff_missedTackle == 1]['nflId'].unique().tolist()
  # Store results in a list
  mtdf = []
  # Create a for loop to iterate through each missed tackler during the play
  for defender in missed_tacklers:
    player = defender
    player_j = temp[temp.nflId == player]['jerseyNumber'].iloc[0]

    min_frame_player = temp[(temp.nflId == defender) & (temp.distance_to_ballcarrier <= 2.0)]['frameId'].min()
    min_time_player = temp[temp.frameId == min_frame_player]['time'].iloc[0]

    # Identify the frame our player made first contact
    if 'first_contact' in temp.event.value_counts().index:
      min_frame_contact = temp[(temp.nflId == defender) & (temp.event == 'first_contact')]['frameId'].min()
      min_time_contact = temp[(temp.nflId == defender) & (temp.event == 'first_contact')]['time'].iloc[0]
    else:
      min_frame_contact = 'N/A'
      min_time_contact = 'N/A'
    # Identify the ball carrier X value at the time of first contact with the missed tackler
    pre_ball_carrier_x = temp[(temp.frameId == min_frame_player)]['ballCarrierX'].iloc[0]

    # Identify the distance of the second closest defender
    next_closest_value = temp[temp['frameId'] == min_frame_player]['distance_to_ballcarrier'].nsmallest(2).iloc[-1]
    next_closest_defender = temp[temp.distance_to_ballcarrier == next_closest_value]['nflId'].iloc[0]
    next_closest_defender_j = temp[temp.distance_to_ballcarrier == next_closest_value]['jerseyNumber'].iloc[0]
    # Identify the frame the ball carrier got passed the defender for 1 yard plus the defenders closest distance
    # Closed distance is determined only 1.5 seconds (15 frames) after the player entered the 2 yard distance zone.
    # Reason for this is when players miss the tackle but catch the ball carrier at the end of the play and get closer
    # than they were during the intial missed tackle throwing off the metrics.
    missed_tackler_closest_dist = temp[(temp.nflId == defender) &
                                      (temp.frameId <= min_frame_player + 15)]['distance_to_ballcarrier'].min()
    mtcd_frame = temp[(temp.nflId == defender) & (temp.distance_to_ballcarrier == missed_tackler_closest_dist)]['frameId'].iloc[0]
    distance_one_test = temp[(temp.frameId > mtcd_frame) &
                            (temp.nflId == player)]['distance_to_ballcarrier'].max()
    if distance_one_test >= missed_tackler_closest_dist+1.0:
      max_frame_player = temp[(temp.frameId > mtcd_frame) &
                              (temp.distance_to_ballcarrier >= missed_tackler_closest_dist+1.0) &
                              (temp.nflId == defender)]['frameId'].min()
    else:
      max_frame_player = temp[(temp.frameId > temp[temp.distance_to_ballcarrier == missed_tackler_closest_dist]['frameId'].iloc[0]) &
                              (temp.distance_to_ballcarrier >= missed_tackler_closest_dist*2) &
                              (temp.nflId == defender)]['frameId'].min()
    max_time_player = temp[temp.frameId == max_frame_player]['time'].iloc[0]

    # Calculate the next closest defender distance gained (add if statement in case it's different)
    max_closest_value = temp[temp['frameId'] == max_frame_player]['distance_to_ballcarrier'].nsmallest(1).iloc[-1]
    max_second_value = temp[temp['frameId'] == max_frame_player]['distance_to_ballcarrier'].nsmallest(2).iloc[-1]
    if temp[temp.distance_to_ballcarrier == max_closest_value]['nflId'].iloc[0] == defender:
      max_closest_player = temp[temp.distance_to_ballcarrier == max_second_value]['nflId'].iloc[0]
      max_closest_player_j = temp[temp.distance_to_ballcarrier == max_second_value]['jerseyNumber'].iloc[0]
      max_closest_value = temp[temp['frameId'] == max_frame_player]['distance_to_ballcarrier'].nsmallest(2).iloc[-1]
    else:
      max_closest_player = temp[temp.distance_to_ballcarrier == max_closest_value]['nflId'].iloc[0]
      max_closest_player_j = temp[temp.distance_to_ballcarrier == max_closest_value]['jerseyNumber'].iloc[0]

    # If offense is driving towards the left then x will be decreasing when the offense is gaining yards
    play_ending_events = ['tackle', 'out_of_bounds', 'qb_slide', 'fumble', 'interception', 'touchdown']
    if offense_pos > defense_pos:
      post_ball_carrier_x = temp[(temp.frameId >= min_frame_player) &
                                (temp.frameId <= temp[temp.event.isin(play_ending_events)]['frameId'].iloc[0])]['ballCarrierX'].min()
      tackler_exit_ball_carrier_x = temp[(temp.frameId == max_frame_player)]['ballCarrierX'].min()
    # If the offense is driving toward the right then x will be increasing when the offense is gaining yards
    else:
      post_ball_carrier_x = temp[(temp.frameId >= min_frame_player) &
                                (temp.frameId <= temp[temp.event.isin(play_ending_events)]['frameId'].iloc[0])]['ballCarrierX'].max()
      tackler_exit_ball_carrier_x = temp[(temp.frameId == max_frame_player)]['ballCarrierX'].max()

    # Calculate the max yards gained after the missed tackler
    if offense_pos < defense_pos:
      yds_gained = post_ball_carrier_x - pre_ball_carrier_x
      missed_yds_gained = tackler_exit_ball_carrier_x - pre_ball_carrier_x
    elif offense_pos > defense_pos:
      yds_gained = pre_ball_carrier_x - post_ball_carrier_x
      missed_yds_gained = pre_ball_carrier_x - tackler_exit_ball_carrier_x
    else:
      yds_gained = 999

    # Calculate time wasted by missed tackler
    time_wasted = (max_time_player - min_time_player).total_seconds()

    results = pd.DataFrame({'missedTackler': defender,
                            'missedTacklerJ': player_j,
                            'enteredFrame': min_frame_player,
                            'enteredTime': min_time_player,
                            'firstContactFrame': min_frame_contact,
                            'firstContactTime': min_time_contact,
                            'nextDefender': next_closest_defender,
                            'nextDefenderJ': next_closest_defender_j,
                            'nextDefenderDistance': next_closest_value,
                            'exitFrame': max_frame_player,
                            'exitTime': max_time_player,
                            'exitDefender': max_closest_player,
                            'exitDefenderJ': max_closest_player_j,
                            'exitDefenderDistance': max_closest_value,
                            'totalTime': time_wasted,
                            'ydsGainedDuringMissedTackle': missed_yds_gained,
                            'totalYdsGainedAfterMissedTackle': yds_gained,
                            'gamePlayId': gamePlayId
                            }, index=[0])
    # Append results to our list
    mtdf.append(results)
  # Concat list results to dataframe
  mtdf = pd.concat(mtdf, ignore_index=True)
  return mtdf

<u>**NOTE:**</u> the collection of functions above will return the key variables in calculating the missed tackle rating score. However, this has to be done for all tracking_week files. Below is the loop that will create the master dataframe iterating through all 9 tracking_week files. This took about an hour to compute.

In [19]:
# Create a function that handles the ETL cleanly
import os
from tqdm import tqdm
# Get all files in the directory
all_files = os.listdir('/kaggle/input/nfl-big-data-bowl-2024')
# Filter files that contain the specified substring
matching_files = [file for file in all_files if "tracking_week" in file]
prefix = '/kaggle/input/nfl-big-data-bowl-2024/'
matching_files = [prefix + element for element in matching_files]
matching_files = sorted(matching_files)

# Define our tackles dim table for all weeks
tackles = tackles_dataset('/kaggle/input/nfl-big-data-bowl-2024/tackles.csv', 
                          '/kaggle/input/nfl-big-data-bowl-2024/players.csv')
# Define our dataset to store the results
mtdf = pd.DataFrame(columns=['missedTackler', 'missedTacklerJ', 'enteredFrame', 'enteredTime', 'firstContactFrame',
                            'firstContactTime','nextDefender', 'nextDefenderJ',
                            'nextDefenderDistance', 'exitFrame', 'exitTime', 'exitDefender', 'exitDefenderJ',
                            'exitDefenderDistance', 'totalTime', 'ydsGainedDuringMissedTackle',
                            'totalYdsGainedAfterMissedTackle', 'gamePlayId'])
# Store list of gamePlayIds with Errors
index_errors = []
value_errors = []

for i in tqdm(matching_files, desc='Executing', unit='iteration'):
  # Add necessary additional columns to make the script work
  tracking = tackle_tracking(enrich_tracking_week(i))
  tt = tackles[['gamePlayId', 'tackle_displayName', 'nflId', 'tackle', 'assist', 'pff_missedTackle']]
  prepped_df = tracking.merge(tt, how='left', on=['gamePlayId', 'nflId'])
  # Create a list, a, that contains all the gamePlayIds that have a missed tackle
  a = prepped_df[prepped_df.pff_missedTackle == 1].gamePlayId.unique().tolist()
  for j in a:
    try:
      input = process_missed_tackles(tracking, tt, j)
      input['sourceFile'] = i
      mtdf = pd.concat([mtdf, input], axis=0, ignore_index=True)
    except IndexError:
      index_errors.append(j)
      print(f"IndexError encountered for gamePlayId: {j}")
      continue
    except ValueError:
      value_errors.append(j)
      print(f"ValueError encountered for gamePlayId: {j}")
      continue

Executing:   0%|          | 0/9 [00:00<?, ?iteration/s]

IndexError encountered for gamePlayId: 20220911035039
IndexError encountered for gamePlayId: 2022091104462
IndexError encountered for gamePlayId: 20220911041020
IndexError encountered for gamePlayId: 20220911042190
IndexError encountered for gamePlayId: 20220911052742
IndexError encountered for gamePlayId: 20220911093087
IndexError encountered for gamePlayId: 2022091110379
IndexError encountered for gamePlayId: 2022091110750
IndexError encountered for gamePlayId: 20220911103492


Executing:  11%|█         | 1/9 [05:14<41:54, 314.32s/iteration]

IndexError encountered for gamePlayId: 20220915001076
IndexError encountered for gamePlayId: 20220918012090
IndexError encountered for gamePlayId: 20220918031412
IndexError encountered for gamePlayId: 2022091805975
IndexError encountered for gamePlayId: 20220918051563
IndexError encountered for gamePlayId: 20220918052361
ValueError encountered for gamePlayId: 20220918052481


Executing:  22%|██▏       | 2/9 [10:03<34:56, 299.49s/iteration]

...

Executing:  78%|███████▊  | 7/9 [32:59<08:55, 267.98s/iteration]

IndexError encountered for gamePlayId: 20221030013194
IndexError encountered for gamePlayId: 2022103002392
IndexError encountered for gamePlayId: 2022103004870
IndexError encountered for gamePlayId: 20221030052279
IndexError encountered for gamePlayId: 20221030054480
IndexError encountered for gamePlayId: 2022103006732
IndexError encountered for gamePlayId: 20221030071459
IndexError encountered for gamePlayId: 20221030081885
IndexError encountered for gamePlayId: 20221030092259
IndexError encountered for gamePlayId: 20221030093022
IndexError encountered for gamePlayId: 20221030111105
IndexError encountered for gamePlayId: 20221030122951


Executing:  89%|████████▉ | 8/9 [37:47<04:34, 274.15s/iteration]

IndexError encountered for gamePlayId: 20221103002580
IndexError encountered for gamePlayId: 20221103003274
IndexError encountered for gamePlayId: 2022110600411
IndexError encountered for gamePlayId: 20221106001461
IndexError encountered for gamePlayId: 20221106002808
IndexError encountered for gamePlayId: 20221106003654
IndexError encountered for gamePlayId: 2022110601739
IndexError encountered for gamePlayId: 20221106011791
IndexError encountered for gamePlayId: 20221106042864
IndexError encountered for gamePlayId: 20221106043047
IndexError encountered for gamePlayId: 20221106051721
IndexError encountered for gamePlayId: 20221106072688
IndexError encountered for gamePlayId: 2022110608339
IndexError encountered for gamePlayId: 2022110608360
IndexError encountered for gamePlayId: 20221106081382
IndexError encountered for gamePlayId: 20221106081535
IndexError encountered for gamePlayId: 20221106093644


Executing: 100%|██████████| 9/9 [41:18<00:00, 275.41s/iteration]


<u>**NOTE:**</u> there are mostly index errors and maybe one or two value errors. Those gamePlayIds are stored in a list. Errors represented less than 3% of plays throughout the dataset. These mostly occurred for one of two reasons:
1. The player missed the tackle but never got within two yards of the ball carrier.
  * Since we are given a dataset without the entire play in most cases, the missed tackle could have occurred at the beginning of the play that was cut out. In that case there is no analysis that can be done.
2. The missed tackler never got further than 1 yard plus the minimum distance away from the ball carrier.
  * In this case, it makes it very difficult to tell at what point the defender missed the tackle. So to prevent potential outliers from getting into the dataset, these were left out.

## Enriching the In-Depth Tackle Data and Calculating the Rating Score

After the `mtdf` dataframe comes out of the for loop, there are few enriching steps that need to take place afterwards:

In [20]:
# Add column to find the distance the defender made up during missed tackle attempt
mtdf['defenderClosingDistance'] = mtdf.nextDefenderDistance - mtdf.exitDefenderDistance
# Add column to find the defender's closing speed
mtdf['defenderClosingSpeed'] = mtdf.defenderClosingDistance / mtdf.totalTime
# Add column ydsGainedDuringMissedTackle as a function of total time
mtdf['inFrameYdsGainedPerSecond'] = mtdf.ydsGainedDuringMissedTackle / mtdf.totalTime

In [21]:
# Define the rating system
from sklearn.preprocessing import StandardScaler

rating_df = mtdf[['gamePlayId', 'missedTackler', 'inFrameYdsGainedPerSecond',
                  'defenderClosingDistance', 'totalYdsGainedAfterMissedTackle']]
std = StandardScaler()
rating_df[['inFrameYdsGainedPerSecond', 'totalYdsGainedAfterMissedTackle', 'defenderClosingDistance']] = std.fit_transform(rating_df[['inFrameYdsGainedPerSecond', 'totalYdsGainedAfterMissedTackle', 'defenderClosingDistance']])
rating_df['inFrameYdsGainedPerSecond'] = rating_df.inFrameYdsGainedPerSecond * -1
rating_df['totalYdsGainedAfterMissedTackle'] = rating_df.totalYdsGainedAfterMissedTackle * -1
rating_df['Rating'] = rating_df.inFrameYdsGainedPerSecond*.5 + rating_df.defenderClosingDistance*.25 + rating_df.totalYdsGainedAfterMissedTackle*.25

/tmp/ipykernel_19/176262457.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df[['inFrameYdsGainedPerSecond', 'totalYdsGainedAfterMissedTackle', 'defenderClosingDistance']] = std.fit_transform(rating_df[['inFrameYdsGainedPerSecond', 'totalYdsGainedAfterMissedTackle', 'defenderClosingDistance']])
/tmp/ipykernel_19/176262457.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df['inFrameYdsGainedPerSecond'] = rating_df.inFrameYdsGainedPerSecond * -1
/tmp/ipykernel_19/176262457.py:9: SettingWi

In [22]:
# Merge ratings
r = mtdf.merge(rating_df[['gamePlayId', 'missedTackler', 'Rating']], how='left', on=['gamePlayId', 'missedTackler'])
# Normalize the ratings so there are no negative values
min_rating = r.Rating.min()
r['Rating'] = r.Rating + min_rating * -1

<u>**NOTE:**</u> at this point, dataframe `r` was exported to excel so that the hour long for loop doesn't have to execute each time. Simply reupload the excel file and continue analyzing the data!